In [1]:
import pandas as pd
from datetime import datetime as dt

In [2]:
data = pd.read_csv("OpenWeatherMap_2017-2019.csv")

In [3]:
365*24

8760

In [4]:
data.shape

(26510, 25)

In [5]:
# check datatypes in columns
data.dtypes

dt                       int64
dt_iso                  object
timezone                 int64
city_name               object
lat                    float64
lon                    float64
temp                   float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
sea_level              float64
grnd_level             float64
humidity                 int64
wind_speed             float64
wind_deg                 int64
rain_1h                float64
rain_3h                float64
snow_1h                float64
snow_3h                float64
clouds_all               int64
weather_id               int64
weather_main            object
weather_description     object
weather_icon            object
dtype: object

In [6]:
# check for missing values in columns
data.isnull().sum()

dt                         0
dt_iso                     0
timezone                   0
city_name                  0
lat                        0
lon                        0
temp                       0
feels_like                 0
temp_min                   0
temp_max                   0
pressure                   0
sea_level              26510
grnd_level             26510
humidity                   0
wind_speed                 0
wind_deg                   0
rain_1h                25414
rain_3h                26510
snow_1h                26507
snow_3h                26510
clouds_all                 0
weather_id                 0
weather_main               0
weather_description        0
weather_icon               0
dtype: int64

As there are a total of 26510 instaces in the dataframe we can summise that features: sea_level, grnd_level, rain_3h and snow_3h are empty

In [7]:
# drop empty features
data.drop(columns=["grnd_level", "sea_level", "rain_3h", "snow_3h"], inplace=True)

In [8]:
# check for duplicated features
temp = data.T
temp.duplicated()

dt                     False
dt_iso                 False
timezone               False
city_name              False
lat                    False
lon                    False
temp                   False
feels_like             False
temp_min               False
temp_max               False
pressure               False
humidity               False
wind_speed             False
wind_deg               False
rain_1h                False
snow_1h                False
clouds_all             False
weather_id             False
weather_main           False
weather_description    False
weather_icon           False
dtype: bool

No duplicate features

In [9]:
# check values for dt
data["dt"].nunique()

26280

In [10]:
print(dt.fromtimestamp(data["dt"].min()))
print(dt.fromtimestamp(data["dt"].max()))

2017-01-01 00:00:00
2019-12-31 23:00:00


In [11]:
start_timestamp = 1514764800   # 01/01/2018 00:00:00
end_timestamp = 1546300800     # 01/01/2019 00:00:00

In [12]:
data = data[data["dt"] >= start_timestamp]
data = data[data["dt"] <= end_timestamp]

In [13]:
print(dt.fromtimestamp(data["dt"].min()))
print(dt.fromtimestamp(data["dt"].max()))

2018-01-01 00:00:00
2019-01-01 00:00:00


In [14]:
data.shape

(8801, 21)

Fewer unique datetime values than unique rows

In [15]:
# check values for dt_iso
data["dt_iso"].nunique()

8761

Matches numer of unique values for feature 'dt'

In [16]:
data.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,humidity,wind_speed,wind_deg,rain_1h,snow_1h,clouds_all,weather_id,weather_main,weather_description,weather_icon
8779,1514764800,2018-01-01 00:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.15,-6.49,3.84,5.79,...,87,12.86,240,NaN,NaN,40,520,Rain,light intensity shower rain,09n
8780,1514768400,2018-01-01 01:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.14,-5.79,3.65,5.86,...,87,11.83,240,NaN,NaN,75,520,Rain,light intensity shower rain,09n
8781,1514772000,2018-01-01 02:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.61,-5.77,3.85,5.99,...,81,12.35,240,NaN,NaN,40,802,Clouds,scattered clouds,03n
8782,1514775600,2018-01-01 03:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.64,-5.73,4.00,6.14,...,81,12.35,240,NaN,NaN,40,802,Clouds,scattered clouds,03n
8783,1514779200,2018-01-01 04:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,5.04,-4.91,4.11,6.22,...,81,11.83,240,NaN,NaN,40,802,Clouds,scattered clouds,03n


In [17]:
count = 0
for i in data.index:
    
    d = dt.strptime(data.loc[i, "dt_iso"][0:-10], "%Y-%m-%d %H:%M:%S").timestamp()
    t = data.loc[i, "dt"]
    
    if d != t:
        if d != (t - 3600):
            print("%s - %s" % (d, t))
            count += 1
        
print(count)

0


Difference of hour in approx 26000 rows - daylight savings?

In [18]:
# check values for timezone
data["timezone"].unique()

array([   0, 3600], dtype=int64)

In [19]:
# check values for city_name
data["city_name"].unique()

array(['Dublin'], dtype=object)

In [20]:
# check values for lat
data["lat"].unique()

array([53.349805])

In [21]:
# check values for lon
data["lon"].unique()

array([-6.26031])

In [22]:
# check min temp value
print(data["temp"].min())
# check max temp value
print(data["temp"].max())

-4.96
25.67


In [23]:
# check min temp value
print(data["feels_like"].min())
# check max temp value
print(data["feels_like"].max())

-16.08
24.77


In [24]:
timestamps = 0
features = []
for i in data["dt"].unique():
    
    df = data[data["dt"] == i]
    l = len(df)
    if l > 1:
        timestamps += l
        for col in df.columns:
            if len(df[col].value_counts()) > 1:
                if col not in features:
                    features.append(col)


In [25]:
timestamps

80

In [26]:
features

['weather_id', 'weather_main', 'weather_description', 'weather_icon']

Some effective duplicate entries (40 duplicated) where all features match (datetime!) except for the weather description

In [27]:
data.columns

Index(['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'temp',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity',
       'wind_speed', 'wind_deg', 'rain_1h', 'snow_1h', 'clouds_all',
       'weather_id', 'weather_main', 'weather_description', 'weather_icon'],
      dtype='object')

In [28]:
data.drop(columns=["temp_min", "temp_max", "rain_1h", "snow_1h", "clouds_all", "weather_id", "weather_icon"], inplace=True)

drop these features:<br>
    temp_min: dropped in favour of average<br>
    temp_max: dropped in favour of average<br>
    rain_1h: predicting based on actual conditions not forecast<br>
    snow_1h: predicting based on actual conditions not forecast<br>
    clouds_all: think unlikely that percentage cloud cover could affect travel times<br>
    weather_id: information specific to 'openweathermaps' system<br>
    weather_icon: information specific to 'openweathermaps' system<br>

In [29]:
data.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,pressure,humidity,wind_speed,wind_deg,weather_main,weather_description
8779,1514764800,2018-01-01 00:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.15,-6.49,990,87,12.86,240,Rain,light intensity shower rain
8780,1514768400,2018-01-01 01:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.14,-5.79,991,87,11.83,240,Rain,light intensity shower rain
8781,1514772000,2018-01-01 02:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.61,-5.77,990,81,12.35,240,Clouds,scattered clouds
8782,1514775600,2018-01-01 03:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.64,-5.73,990,81,12.35,240,Clouds,scattered clouds
8783,1514779200,2018-01-01 04:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,5.04,-4.91,990,81,11.83,240,Clouds,scattered clouds


In [30]:
data["timezone"].value_counts()

3600    5219
0       3582
Name: timezone, dtype: int64

In [31]:
data["dt"].nunique()

8761

In [32]:
data["dt_iso"].nunique()

8761

In [33]:
data.shape

(8801, 14)

In [34]:
data.columns

Index(['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'temp',
       'feels_like', 'pressure', 'humidity', 'wind_speed', 'wind_deg',
       'weather_main', 'weather_description'],
      dtype='object')

Dropping features:<br>
dt_iso: basically duplicates info from dt but in a different format<br>
city_name: redundant, we know his is dublin & all rows are the same<br>
lat: redundant, we know his is dublin & all rows are the same<br>
lon: redundant, we know his is dublin & all rows are the same<br>
pressure: unlikely to direclty impact travel times<br>
himidity: unlikely to direclty impact travel times<br>

In [35]:
data.drop(columns=["dt_iso", "city_name", "lat", "lon", "pressure", "humidity"], inplace=True)

In [36]:
data.to_csv("open_weather_cleaned.csv", index=False)